<a href="https://colab.research.google.com/github/elianedp/ExpertSystem/blob/main/%5BONLINE%5D_Exemplo_1_Chatbot_baseado_em_regras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplo 1 - Chatbot de atendimento ao cliente baseado em regras
## Tecnólogo em Inteligência Artificial Aplicada - Agentes Conversacionais
Neste notebook iremos construir um chatbot utilizando-se das técnicas mais simples de desenvolvimento.

### Chatbot baseado em regras

É o tipo de chatbot mais simples que existe, que consiste em um dataset com respostas pré-definidas e uma série de regras que servem para encontrar estas respostas (ou intenções) no dataset.

Apesar das limitações, eles podem ser bastante eficientes e produtivos se tiverem regras bem definidas e um dataset substancial de respostas.


### Qual o contexto de nosso chatbot?

Iremos desenvolver um chatbot de interação com clientes de um restaurante. O objetivo é que o chatbot auxilie o cliente a encontrar as informações desejadas sem que precise navegar por vários links do site.

### Quais ferramentas e técnicas iremos utilizar?

*   **NLTK** - O mais famoso toolkit de Processamento de Linguagem Natural em Python.
*   **Expressões Regulares** - o pacote de regex do Python será utilizado para otimizar a busca de padrões.
*   **WordNet** - o grande banco de dados léxico e semântico, disponível em diversos idiomas.



### Construindo o chatbot
Nosso chatbot vai operar da seguinte maneira:

1.   Recebe **entrada** do usuário
2.   Procura **palavras-chave** na entrada do usuário
3.   Define a **intenção associada** à entrada
4.   Obtém a **resposta baseada na intenção** definida
5.   Mostra a **resposta** ao usuário

Portanto, temos de ter uma **tabela que associe as palavras-chave desejadas, com as intenções definidas** em nosso dataset. A seguir um exemplo:
![Exemplo de busca de intenção e resposta](https://docs.google.com/uc?export=download&id=1LnOTZiahBSv9VGrCSExbylNbxHpclMKZ)

#### Importando as bibliotecas
Vamos importar o pacote de expressões regulares do Python e também o acesso ao WordNet dado pelo NLTK.

#### Construindo a lista de palavras-chave
Esta é uma etapa que pode demandar bastante tempo, principalmente de você quiser definir as palavras manualmente. Porém, quanto mais palavras-chave você tiver, maior cobertura seu chatbot vai ter.

Por questões didáticas, vamos a partir de um único termo, tentar automatizar esta busca, utilizando os sinônimos encontrados no WordNet.

In [1]:
# Instala a biblioteca unidecode (só precisa rodar uma vez no ambiente)
!pip install unidecode

import re
import nltk
from nltk.corpus import wordnet
import unidecode  # para remover acentos

# Baixar pacotes necessários
nltk.download('wordnet')
nltk.download('omw-1.4')

# Lista de palavras
palavras = ["olá", "horário", "cardápio"]
lista_sinonimos = {}

for palavra in palavras:
    sinonimos = []
    for syn in wordnet.synsets(palavra, lang="por"):
        for nome in syn.lemma_names("por"):
            sinonimos.append(unidecode.unidecode(nome.lower()))
    lista_sinonimos[palavra] = set(sinonimos)

print("Lista de sinônimos gerada:")
print(lista_sinonimos)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Lista de sinônimos gerada:
{'olá': {'ola'}, 'horário': {'ordem_do_dia', 'horario', 'plano', 'programa', 'agenda', 'cronograma'}, 'cardápio': {'cardapio', 'menu'}}


In [2]:
# Lista de palavras-chave (intenções que o chatbot deve reconhecer)
# Já incluí "olá", "horário" e adicionamos "cardápio" como melhoria
palavras = ["olá", "horário", "cardápio"]

# Dicionário de sinônimos
lista_sinonimos = {}

# Percorre a lista de palavras
for palavra in palavras:
    sinonimos = []
    # Busca sinônimos da palavra no WordNet (em português)
    for syn in wordnet.synsets(palavra, lang="por"):
        for lem in syn.lemma_names("por"):
            # Normaliza: minúsculas e sem acento
            sinonimos.append(unidecode.unidecode(lem.lower()))
    # Remove duplicatas e adiciona ao dicionário
    lista_sinonimos[palavra] = set(sinonimos)

# Mostra os sinônimos encontrados
print("Lista de sinônimos gerada:")
print(lista_sinonimos)

Lista de sinônimos gerada:
{'olá': {'ola'}, 'horário': {'ordem_do_dia', 'horario', 'plano', 'programa', 'agenda', 'cronograma'}, 'cardápio': {'cardapio', 'menu'}}


#### Construindo dicionário de intenções
Vamos utilizar o dicionário de sinônimos construído e mapeá-los para as possíveis intenções. Além disso, vamos formatar as palavras-chave de modo que sejam interpretadas pela ferramenta de busca de expressões regulares.

In [3]:
# Construindo dicionário de intenções a partir dos sinônimos
keywords = {}
keywords_dict = {}

# 🔹 Intenção: Saudação
keywords['saudacao'] = []
for sin in list(lista_sinonimos['olá']):
    keywords['saudacao'].append(r'.*\b' + sin + r'\b.*')

# 🔹 Intenção: Horário de atendimento
keywords['horario_atendimento'] = []
for sin in list(lista_sinonimos['horário']):
    keywords['horario_atendimento'].append(r'.*\b' + sin + r'\b.*')

# 🔹 Intenção: Cardápio (melhoria que eu adicionei)
keywords['cardapio'] = []
for sin in list(lista_sinonimos['cardápio']):
    keywords['cardapio'].append(r'.*\b' + sin + r'\b.*')

# 🔹 Intenção: Endereço (adicionei manualmente)
keywords['endereco'] = [r'.*\bendereco\b.*', r'.*\bruab\b.*', r'.*\bloc\b.*']

# 🔹 Intenção: Delivery (adicionei manualmente)
keywords['delivery'] = [r'.*\bdelivery\b.*', r'.*\bentrega\b.*', r'.*\bpedido\b.*']

# 🔹 Intenção: Reserva de mesa (adicionei manualmente)
keywords['reserva'] = [r'.*\breserva\b.*', r'.*\bmesa\b.*', r'.*\breservar\b.*']

# Compila os padrões regex para cada intenção
for intent, keys in keywords.items():
    keywords_dict[intent] = re.compile('|'.join(keys))

# Visualiza o dicionário de intenções
print("Dicionário de intenções (regex):")
print(keywords_dict)

Dicionário de intenções (regex):
{'saudacao': re.compile('.*\\bola\\b.*'), 'horario_atendimento': re.compile('.*\\bordem_do_dia\\b.*|.*\\bhorario\\b.*|.*\\bplano\\b.*|.*\\bprograma\\b.*|.*\\bagenda\\b.*|.*\\bcronograma\\b.*'), 'cardapio': re.compile('.*\\bcardapio\\b.*|.*\\bmenu\\b.*'), 'endereco': re.compile('.*\\bendereco\\b.*|.*\\bruab\\b.*|.*\\bloc\\b.*'), 'delivery': re.compile('.*\\bdelivery\\b.*|.*\\bentrega\\b.*|.*\\bpedido\\b.*'), 'reserva': re.compile('.*\\breserva\\b.*|.*\\bmesa\\b.*|.*\\breservar\\b.*')}


In [6]:
# Aqui eu estou criando as respostas que o meu chatbot vai usar.
# Também deixei uma resposta padrão para quando ele não entender a pergunta.
# Além disso, adicionei novas intenções: endereço, delivery e reserva de mesa.

respostas = {
    'saudacao': 'Olá! Como posso ajudá-lo?',
    'horario_atendimento': 'Abrimos para almoço das 11h30 às 15h00 e para jantar das 18h30 às 23h00.',
    'cardapio': 'Nosso cardápio inclui pratos variados, como massas, saladas e carnes.',  # melhoria que eu adicionei
    'endereco': 'Nosso restaurante fica na Rua das Flores, nº 123, no centro da cidade.',
    'delivery': 'Sim, fazemos delivery! Você pode pedir pelo nosso site ou pelo telefone (45) 1234-5678.',
    'reserva': 'Para reservar uma mesa, por favor me diga a data, o horário e o número de pessoas.',
    'padrao': 'Me desculpe, mas não entendi o que você disse. Você poderia reformular?'
}


> **NOTA**: Aqui poderíamos ainda criar mais de uma resposta para cada intenção, evitando que a interação fique repetitivo.




#### Linkando as intenções e gerando as repostas (front-end)
Aqui construíremos de fato o algoritmo que fará a interação com o usuário (o gerenciador de diálogo), onde receberemos uma entrada do mesmo, e utilizando regex, iremos buscar as palavras-chave no texto e associar com a intenção desejada.

In [ ]:
print("Bem-vindo ao restaurante McChat!")

# Dicionário para salvar reservas feitas
reservas = []

# Roda indefinidamente, até que o usuário peça para sair
while True:

    # Pega o input do usuário e normaliza em letras minúsculas
    entrada = input("Você: ").lower()

    # Define condição de saída
    if entrada == 'sair':
        print("Chatbot: Obrigado pela visita. Volte sempre!")
        break

    matched_intent = None

    # Verifica se alguma intenção bate com a entrada
    for intent, pattern in keywords_dict.items():
        if re.search(pattern, entrada):
            matched_intent = intent
            break  # para na primeira correspondência encontrada

    # Caso especial: intenção de reserva
    if matched_intent == 'reserva':
        # Tenta identificar quantidade de pessoas
        pessoas = re.findall(r'\d+', entrada)
        qtd = pessoas[0] if pessoas else "não informado"

        # Salva a reserva em uma lista de dicionários
        reservas.append({
            "pedido": entrada,
            "pessoas": qtd
        })

        print(f"Chatbot: Ok, anotei uma reserva para {qtd} pessoa(s).")
        continue  # volta para o próximo input

    # Por padrão, usamos a intenção 'padrao'
    key = 'padrao'
    if matched_intent in respostas:
        key = matched_intent

    # O chatbot imprime a resposta correspondente
    print("Chatbot:", respostas[key])

Bem-vindo ao restaurante McChat!
Você: cardapio
Chatbot: Nosso cardápio inclui pratos variados, como massas, saladas e carnes.


Assim, finalizamos o desenvolvimento de nosso modelo de chatbot.


### O que pode ser feito?
Como dito inicialmente, esta é a abordagem e arquitetura mais simples de chatbot, que depende muito do tamanho e cobertura da base de dados para seu sucesso.
Então, o que poderíamos fazer para melhorar?


*   Inclua mais intenções na base de dados relacionadas a dados necessários para um restaurante (e.g., endereço, cardápio, delivery, reservas)
*   Tente fazer uma regra específica para reserva de mesas no restaurante em que se salve as reservas em um dicionário e utilize regex para obter os parâmetros de data, horário e quantidade de pessoas em meio ao texto
*   Caso o restaurante abra para almoço e jantar, como resolver a intenção de horário de atendimento?



## Referências e Material complementar

* [Criando meu ChatBot com Python](https://medium.com/@erikatiliorey/criando-um-chatbot-com-python-36f24b62df6c)
* [Introducing Conversational Chat bots using Rule Based Approach](https://chatbotslife.com/introducing-conversational-chat-bots-using-rule-based-approach-c8840aeaad07)
* [Building a Simple Chatbot from Scratch in Python (using NLTK)](https://medium.com/analytics-vidhya/building-a-simple-chatbot-in-python-using-nltk-7c8c8215ac6e)
* [How to write a smart Chatbot in 40 lines without ML](https://levelup.gitconnected.com/a-chatbot-without-machine-learning-b76ad00e30c1)
* [Building a Rule-based Chatbot in Python](https://blog.datasciencedojo.com/building-a-rule-based-chatbot-in-python/)
* [Basic FAQ chat bot](https://github.com/canonicalmg/FAQ-Chat-Bot)

Este notebook foi produzido por Prof. [Lucas Oliveira](http://lattes.cnpq.br/3611246009892500).